In [2]:
!pip install langchain                # Core
!pip install langchain-community      # Community loaders & tools (PyPDFLoader etc.)
!pip install langchain-qdrant         # Qdrant integration
!pip install langchain-huggingface    # HuggingFace embeddings
!pip install langchain-cohere

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 34.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 444.0/444.0 kB 25.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.3 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.32.4
    Uninstalling requests-2.32.4:
      Successfully uninstalled requests-2.32.4
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.74
    Uninstalling langchain-core-0.3.74:
      Successfully uninstalled langchain-core-0.3.74
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 337.3/337.3 kB 7.8 

In [10]:
!pip install pypdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.5/310.5 kB 5.3 MB/s eta 0:00:00


In [19]:
!pip install qdrant-client


In [27]:
!pip install sentence-transformers

In [3]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_qdrant import QdrantVectorStore
from langchain_huggingface import HuggingFaceEmbeddings

In [41]:
from langchain_core.prompts import ChatPromptTemplate

In [4]:
from langchain_cohere import ChatCohere

In [43]:
from qdrant_client import QdrantClient
from qdrant_client.http.models import Distance, VectorParams


In [45]:
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain

In [11]:
from langchain_community.document_loaders import PyPDFLoader

In [28]:
from sentence_transformers import SentenceTransformer

In [6]:
import pandas as pd

In [7]:
loader = PyPDFLoader("/content/Ravindra Balaji gen CV Updated.pdf")

In [12]:
docs = loader.load()

In [13]:
docs

[Document(metadata={'producer': 'Microsoft® Word 2021', 'creator': 'Microsoft® Word 2021', 'creationdate': '2025-07-02T10:41:50+05:30', 'author': 'Gaurav Bali', 'moddate': '2025-07-02T10:41:50+05:30', 'source': '/content/Ravindra Balaji gen CV Updated.pdf', 'total_pages': 1, 'page': 0, 'page_label': '1'}, page_content="RAVINDRA BALAJI NAGULA \nLinkedIn : www.linkedin.com/in/ravindrabalaji/                                                                                  Email:ravindrabalaji.nagula@gmail.com \nGithub   : github.com/Ravindra-balaji                                                                                                          Mobile: +91-9515175300 \n \nSKILLS \n• Languages:  Java, Python, C, C++ \n• Frameworks:  Scikit-Learn, Keras, TensorFlow, Flask \n• Libraries:    NumPy, Pandas, Matplotlib, Seaborn, Ultralytics (YOLO), OpenCV, Pickle, NLTK \n• Tools &Environments: Power-BI, MySQL, Google Colab, Jupyter Notebook \n• Soft Skills:  Problem-Solving Skills, Team 

In [14]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=100
)
chunks = text_splitter.split_documents(docs)

In [16]:
model_name="sentence-transformers/all-mpnet-base-v2"
#model_name="BAAI/bge-large-en-v1.5"
#model_name="nomic-ai/nomic-embed-text-v1.5"

encode_kwargs = {'normalize_embeddings': False}
embeddings = HuggingFaceEmbeddings(model_name=model_name,encode_kwargs=encode_kwargs)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [17]:
path_name = "../tmp/langchain_qdrant3"
#path_name = "../embeddings/nomic_ai"

collection_name = "my_cv"

In [21]:
client = QdrantClient(path=path_name)

In [22]:
client.create_collection(
    collection_name=collection_name,
    vectors_config=VectorParams(size=768, distance=Distance.COSINE),
)

True

In [23]:
vector_store = QdrantVectorStore(
    client=client,
    collection_name=collection_name,
    embedding=embeddings,
)

In [24]:
vector_store.add_documents(documents=chunks)

['8c5118854fcb4e66847c5bf8cd923fd5',
 'eb1ca6a244e94ab2b624594062d4ef93',
 'dc01567f80bb44a29b9f99394185dfbc',
 'cf24eeaf8f9e43cbb6a2c7368d2b81d7',
 '1abde64c5e954f5e8afe6422f40bbd27',
 '5a76d6d7ae83429986d6466607ba46de',
 'ce039980966747de8eaf34292bbb93dc',
 'b4dee748fd8a41f9a2f002c5709e9c86',
 '6d5b375a7aee4ab3bf424cf44564e19a',
 '797a19af1a1e43c69f36350b1ffb630d']

In [29]:
model = SentenceTransformer("sentence-transformers/all-mpnet-base-v2")

In [30]:
print(client.get_collections())

collections=[CollectionDescription(name='my_cv')]


In [31]:
retriever = vector_store.as_retriever(search_type='mmr',
                          search_kwargs={'k':2}
                         )

In [33]:
vector_store = QdrantVectorStore(
    client=client,
    collection_name=collection_name,
    embedding=embeddings,
)

In [34]:
retriever = vector_store.as_retriever(search_type='mmr',
                          search_kwargs={'k':10}
                         )

In [35]:
user_query = "Machine Learning"

In [36]:
retrieval = retriever.get_relevant_documents(user_query)

/tmp/ipython-input-2066677992.py:1: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  retrieval = retriever.get_relevant_documents(user_query)


In [37]:
retriever.invoke(user_query)

[Document(metadata={'producer': 'Microsoft® Word 2021', 'creator': 'Microsoft® Word 2021', 'creationdate': '2025-07-02T10:41:50+05:30', 'author': 'Gaurav Bali', 'moddate': '2025-07-02T10:41:50+05:30', 'source': '/content/Ravindra Balaji gen CV Updated.pdf', 'total_pages': 1, 'page': 0, 'page_label': '1', '_id': 'ce039980966747de8eaf34292bbb93dc', '_collection_name': 'my_cv'}, page_content='• Fundamentals of Management by Coursera.                                               March 2023 \n• Getting Started with AI and Machine Learning by LinkedIn Learning.        February 2023 \n \nACHIEVEMENTS  \nSecured 2nd   position on behalf of our state (AP) in One India event at LPU university                                        April 2023                    \n   For excellent performance in extra-curricular activities and cultural showcase on behalf of the State at University.'),
 Document(metadata={'producer': 'Microsoft® Word 2021', 'creator': 'Microsoft® Word 2021', 'creationdate': '2025-

In [38]:
llm = ChatCohere(
    model="command-r-plus",   # or "command-r", "command", etc.
    temperature=0.3,
    max_tokens=300,
    cohere_api_key="dRrV13nDT200p0mV7g3hKyBfmgIAf2YWxspzO4cl"
)

In [39]:
llm.invoke('hey')

AIMessage(content='Hello! How can I help you today?', additional_kwargs={'id': '63d93f3a-49c3-4ae9-b83d-70b5197c0a36', 'finish_reason': 'COMPLETE', 'content': 'Hello! How can I help you today?', 'token_count': {'input_tokens': 67.0, 'output_tokens': 9.0}}, response_metadata={'id': '63d93f3a-49c3-4ae9-b83d-70b5197c0a36', 'finish_reason': 'COMPLETE', 'content': 'Hello! How can I help you today?', 'token_count': {'input_tokens': 67.0, 'output_tokens': 9.0}}, id='run--da06f477-79af-4281-98f2-a8fbde603a12-0', usage_metadata={'input_tokens': 67, 'output_tokens': 9, 'total_tokens': 76})

In [50]:
prompt = ChatPromptTemplate.from_template("""You are given a CV/resume snippets from a candidate.
Your task is to extract and summarize only the information explicitly present in the CV that is relevant to the question.

- Only use the provided CV text.
- Do not add assumptions or external knowledge.
- Do not infer skills, roles, or qualifications that are not clearly mentioned.
- Do not generate career advice or commentary.
- Answer strictly based on the CV content.

Answer the following question based only on the provided context:

<context>
{context}
</context>

Question: {input}""")

In [51]:
document_chain = create_stuff_documents_chain(llm, prompt)

In [52]:
retrieval_chain = create_retrieval_chain(retriever, document_chain)

In [55]:
user_query = " what are some Datascience projects in the cv"

In [56]:
response = retrieval_chain.invoke({"input": user_query})


In [57]:
print(response["answer"])

- Multi-Persona Chatbot: An AI chatbot project that responds differently based on selected roles like Doctor, Teacher, or Tech Support. The project utilized Cohere's language model with a Flask backend and a user-friendly frontend to generate context-aware responses. 

- Automatic Drone Detection System: The goal was to detect unauthorized drones in image/video sources. The candidate trained a predictive model using YOLOv8 (Ultralytics) with drone images, achieving 94.73% mAP@0.5 and 71.19% mAP@0.5:0.95 accuracy for drone detection. 

- Twitter Sentiment Analysis: This project aimed to predict the sentiment of tweets (positive or negative) using 1.6M tweets. The candidate developed a predictive model using XG-Boost, Logistic Regression, and NLP techniques, achieving 90% accuracy.
